# Chapter 19. Concurrency Models in Python

In [1]:
import itertools
import time
from threading import Thread, Event

In [2]:
def spin(msg: str, done: Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
    time.sleep(3)
    return 42

In [3]:
def supervisor() -> int:
    done=Event()
    spinner = Thread(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result


In [4]:
result = supervisor()
print(result)

spinner object: <Thread(Thread-4 (spin), initial)>
42          
